In [5]:
import numpy as np 
import os
import pandas as pd 
import time
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import tree
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn import metrics
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.neighbors import KNeighborsClassifier 
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [2]:
pip install seaborn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 4.7 MB/s eta 0:00:00 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install imbalanced-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.6/235.6 kB 1.5 MB/s eta 0:00:00m eta 0:00:010:0101
Note: you may need to restart the kernel to use updated packages.


In [6]:

import os
for dirname, _, filenames in os.walk('/home/pranav/Desktop/ML/Project/archive/MachineLearningCSV/MachineLearningCVE'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/home/pranav/Desktop/ML/Project/archive/MachineLearningCSV/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv
/home/pranav/Desktop/ML/Project/archive/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
/home/pranav/Desktop/ML/Project/archive/MachineLearningCSV/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv
/home/pranav/Desktop/ML/Project/archive/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
/home/pranav/Desktop/ML/Project/archive/MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
/home/pranav/Desktop/ML/Project/archive/MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
/home/pranav/Desktop/ML/Project/archive/MachineLearningCSV/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv
/home/pranav/Desktop/ML/Project/archive/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv


In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
low_memory=False
df1 = pd.read_csv("/home/pranav/Desktop/ML/Project/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df2=pd.read_csv("/home/pranav/Desktop/ML/Project/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df3=pd.read_csv("/home/pranav/Desktop/ML/Project/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv")
df4=pd.read_csv("/home/pranav/Desktop/ML/Project/MachineLearningCSV/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv")
df5=pd.read_csv("/home/pranav/Desktop/ML/Project/MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
df6=pd.read_csv("/home/pranav/Desktop/ML/Project/MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
df7=pd.read_csv("/home/pranav/Desktop/ML/Project/MachineLearningCSV/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv")
df8=pd.read_csv("/home/pranav/Desktop/ML/Project/MachineLearningCSV/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv")

In [8]:
df=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8])

In [9]:
df[' Label'].value_counts()

 Label
BENIGN                        2273097
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64

In [10]:
df.shape


(2830743, 79)

In [13]:
# Remove duplicate rows
df = df.drop_duplicates(keep="first")

# Remove rows with NaN, positive infinity, or negative infinity
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)]

In [14]:
df.shape

(2520798, 79)

In [17]:
# Convert int64 into int32 and float64 into float32
integer = []
f = []
for i in df.columns[:-1]:
    if df[i].dtype == "int64":
        integer.append(i)
    elif df[i].dtype == "float64":
        f.append(i)

df[integer] = df[integer].astype("int32")
df[f] = df[f].astype("float32")


In [18]:
# Exclude non-numeric columns
numeric_df = df.select_dtypes(include=['int32', 'float32'])

# Calculate the correlation matrix for numeric columns
corr_matrix = numeric_df.corr()

# Define the correlation threshold
threshold = 0.85

# Find highly correlated features
corr_features = set()
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:
            colname = corr_matrix.columns[i]
            corr_features.add(colname)

# Drop highly correlated features from the original DataFrame
df.drop(corr_features, axis=1, inplace=True)


In [19]:
df.shape

(2520798, 44)

In [20]:
df[' Label'].value_counts()

 Label
BENIGN                        2095057
DoS Hulk                       172846
DDoS                           128014
PortScan                        90694
DoS GoldenEye                   10286
FTP-Patator                      5931
DoS slowloris                    5385
DoS Slowhttptest                 5228
SSH-Patator                      3219
Bot                              1948
Web Attack � Brute Force         1470
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64

In [31]:
#Create a new dataframe with the attacks that have more than 10000 instances in the original dataframe
df1=df.groupby(' Label').filter(lambda x:len(x)>10000)
df1[' Label'].value_counts()

 Label
BENIGN           2095057
DoS Hulk          172846
DDoS              128014
PortScan           90694
DoS GoldenEye      10286
Name: count, dtype: int64

In [30]:
#Create a new dataframe with the attacks that have less than 10000 instances in the original dataframe
df2=df.groupby(' Label').filter(lambda x:len(x)<10000)
df2[' Label'].value_counts()

 Label
FTP-Patator                   5931
DoS slowloris                 5385
DoS Slowhttptest              5228
SSH-Patator                   3219
Bot                           1948
Web Attack � Brute Force      1470
Web Attack � XSS               652
Infiltration                    36
Web Attack � Sql Injection      21
Heartbleed                      11
Name: count, dtype: int64

In [24]:
x = df1.drop([' Label'],axis=1)
y = df1[' Label']

In [25]:
unique, countsa = np.unique(y, return_counts=True)
print(list(zip(unique,countsa)))

[('BENIGN', 2095057), ('DDoS', 128014), ('DoS GoldenEye', 10286), ('DoS Hulk', 172846), ('PortScan', 90694)]


In [29]:
#Down sample the first dataframe (attacks with more than 10000) so that each calss has around 10000 instances. A random undersampling is oerformed for the purpose.
rus = RandomUnderSampler(random_state=0)
rus.fit(x, y)
Xn, yn = rus.fit_resample(x, y)

In [27]:
unique, countsb = np.unique(yn, return_counts=True)
print(list(zip(unique,countsb)))

[('BENIGN', 10286), ('DDoS', 10286), ('DoS GoldenEye', 10286), ('DoS Hulk', 10286), ('PortScan', 10286)]


In [34]:
df1=pd.concat([Xn,yn],axis=1)
df1[' Label'].value_counts()

 Label
BENIGN           10286
DDoS             10286
DoS GoldenEye    10286
DoS Hulk         10286
PortScan         10286
Name: count, dtype: int64

In [35]:
df=pd.concat([df1,df2])
df[' Label'].value_counts()

 Label
BENIGN                        10286
DDoS                          10286
DoS GoldenEye                 10286
DoS Hulk                      10286
PortScan                      10286
FTP-Patator                    5931
DoS slowloris                  5385
DoS Slowhttptest               5228
SSH-Patator                    3219
Bot                            1948
Web Attack � Brute Force       1470
Web Attack � XSS                652
Infiltration                     36
Web Attack � Sql Injection       21
Heartbleed                       11
Name: count, dtype: int64

In [36]:
x = df.drop([' Label'],axis=1)
y = df[' Label']

In [39]:
#Change the categorical labels into numerical
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
le = LabelEncoder()
y=le.fit_transform(df[' Label'].values)

In [40]:
unique, counts1 = np.unique(y, return_counts=True)
print(list(zip(unique,counts1)))

[(0, 10286), (1, 1948), (2, 10286), (3, 10286), (4, 10286), (5, 5228), (6, 5385), (7, 5931), (8, 11), (9, 36), (10, 10286), (11, 3219), (12, 1470), (13, 21), (14, 652)]


In [41]:
#Oversampling with SMOTE to balance the dataset
from imblearn.over_sampling import SMOTE
#strategy ={0:16805,1:16805,2:16805,3:16805,4:16805,5:16805,6:16805,7:16805,8:16805,9:16805,10:16805,11:16805,12:16805,13:16805,14:16805}
oversample = SMOTE(random_state=42)
Xn,Yn = oversample.fit_resample(x, y)

In [42]:
unique, counts = np.unique(Yn, return_counts=True)
print(list(zip(unique,counts)))

[(0, 10286), (1, 10286), (2, 10286), (3, 10286), (4, 10286), (5, 10286), (6, 10286), (7, 10286), (8, 10286), (9, 10286), (10, 10286), (11, 10286), (12, 10286), (13, 10286), (14, 10286)]


In [43]:
X_train, X_test, Y_train, Y_test = train_test_split(Xn,Yn,test_size=0.30,random_state=0)

In [44]:
from sklearn.impute import SimpleImputer

# Create an instance of SimpleImputer with 'mean' strategy to replace NaN values
imputer = SimpleImputer(strategy='mean')

# Fit the imputer to X_train and transform X_train and X_test with it
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Apply StandardScaler to X_train and X_test
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [45]:
start = time.time()
KNN = KNeighborsClassifier(n_neighbors=3)
KNN.fit(X_train, Y_train) 
print("Time taken to train model: ", time.time()-start," seconds")

Time taken to train model:  0.01997661590576172  seconds


In [46]:
from sklearn import metrics
Predict_X =  KNN.predict(X_train)
scores = cross_val_score(KNN, X_train, Y_train, cv=7)
accuracy = metrics.accuracy_score(Y_train,Predict_X)
confusion_matrix = metrics.confusion_matrix(Y_train, Predict_X)
classification = metrics.classification_report(Y_train, Predict_X)

In [47]:
from sklearn import metrics
Predict_X =  KNN.predict(X_train)
scores = cross_val_score(KNN, X_train, Y_train, cv=7)
accuracy = metrics.accuracy_score(Y_train,Predict_X)
confusion_matrix = metrics.confusion_matrix(Y_train, Predict_X)
classification = metrics.classification_report(Y_train, Predict_X)

In [49]:
#Performance 
print('--------------------------- Results OF TEST--------------------------------')
Predict_X = KNN.predict(X_test)
#scores = cross_val_score(KNN, X_test, Y_test, cv=7)
print(f"Cross Validation Accuracy Score: {scores.mean() * 100:.2f}%")
print(f"Accuracy Score: {metrics.accuracy_score(Y_test,Predict_X) * 100:.2f}%")
print(f"F1 Score: {metrics.f1_score(Y_test,Predict_X, average='weighted') * 100:.2f}%")
print(f"Recall Score: {metrics.recall_score(Y_test,Predict_X, average='weighted') * 100:.2f}%")
print(f"Precision Score: {metrics.precision_score(Y_test,Predict_X, average='weighted') * 100:.2f}%")
classification = metrics.classification_report(Y_test, Predict_X)
print("Classification report:" "\n", classification) 
print(f"F1 Macro Score: {metrics.f1_score(Y_test,Predict_X, average='macro') * 100:.2f}%")
print(f"F1 Micro Score: {metrics.f1_score(Y_test,Predict_X, average='micro') * 100:.2f}%")


--------------------------- Results OF TEST--------------------------------
Cross Validation Accuracy Score: 96.43%
Accuracy Score: 96.47%
F1 Score: 96.45%
Recall Score: 96.47%
Precision Score: 96.44%
Classification report:
               precision    recall  f1-score   support

           0       0.99      0.95      0.97      3075
           1       0.98      1.00      0.99      3114
           2       1.00      1.00      1.00      3044
           3       0.99      1.00      1.00      3025
           4       0.99      1.00      0.99      3111
           5       0.99      0.99      0.99      3076
           6       0.99      0.99      0.99      3080
           7       1.00      1.00      1.00      3043
           8       1.00      1.00      1.00      3092
           9       0.99      1.00      1.00      3135
          10       0.99      1.00      0.99      3081
          11       0.99      0.99      0.99      3151
          12       0.79      0.77      0.78      3110
          13      